In [ ]:
import zipfile
import tensorflow as tf
import os
#import gdown
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from PIL import Image
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import numpy as np
import shutil
import random

In [ ]:
#!rm -rf aves

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

zip_path = '/content/drive/My Drive/si.zip'  # Cambia esta ruta si tu archivo está en otro lugar de tu Drive
extract_path = '/content/aves'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verificar la estructura de carpetas
for root, dirs, files in os.walk(extract_path):
    print(root, dirs, files)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/aves ['PROYECTO'] []
/content/aves/PROYECTO ['comrav', 'eursta', 'sonspa', 'houwre', 'houspa', 'bncfly', 'spotow', 'redcro', 'gbwwre1', 'norcar'] []
/content/aves/PROYECTO/comrav ['class_a'] []
/content/aves/PROYECTO/comrav/class_a [] ['XC358147_segment_90.png', 'XC560493_segment_2.png', 'XC295272_segment_2.png', 'XC385374_segment_16.png', 'XC338430_segment_25.png', 'XC549438_segment_9.png', 'XC390197_segment_13.png', 'XC358072_segment_85.png', 'XC593330_segment_1.png', 'XC130898_segment_5.png', 'XC611489_segment_6.png', 'XC130433_segment_19.png', 'XC390236_segment_27.png', 'XC205528_segment_41.png', 'XC358148_segment_117.png', 'XC167827_segment_2.png', 'XC494102_segment_6.png', 'XC278922_segment_1.png', 'XC246433_segment_21.png', 'XC533177_segment_4.png', 'XC374528_segment_2.png', 'XC414012_segment_10.png', 'XC388855_segment_9.png', 'XC170400_segmen

In [ ]:
data_directory = '/content/aves/PROYECTO'
included_categories = ['comrav', 'bncfly', 'eursta', 'gbwwre1', 'houspa', 'houwre', 'norcar', 'redcro', 'sonspa', 'spotow']
selected_directory = '/content/selected_aves'
os.makedirs(selected_directory, exist_ok=True)

In [ ]:
for category in included_categories:
    category_path = os.path.join(data_directory, category, 'class_a')
    selected_category_path = os.path.join(selected_directory, category)
    os.makedirs(selected_category_path, exist_ok=True)

    images = [img for img in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, img))]
    if len(images) >= 4550:
        selected_images = np.random.choice(images, 4550, replace=False)
    else:
        selected_images = images

    for img in selected_images:
        src = os.path.join(category_path, img)
        dst = os.path.join(selected_category_path, img)
        shutil.copyfile(src, dst)

In [ ]:
def convert_resize_and_normalize(img_path, target_size=(224, 224)):
    img = Image.open(img_path)
    img = img.convert('RGB')  # Convertir a RGB si la imagen tiene canal alfa (RGBA)
    img = img.resize(target_size, Image.LANCZOS)  # Redimensionar la imagen a 224x224
    img_array = np.array(img)  # Convertir a numpy array
    img_array = img_array / 255.0  # Normalizar al rango [0, 1]
    img_array = (img_array * 255).astype(np.uint8)  # Convertir de nuevo a uint8
    img_normalized = Image.fromarray(img_array)  # Convertir de nuevo a imagen PIL
    img_normalized.save(img_path)  # Sobrescribir la imagen original

def process_images_in_folder(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.png'):  # Asumiendo que las imágenes son archivos PNG
                img_path = os.path.join(root, file)
                convert_resize_and_normalize(img_path)
                print(f"Procesando {img_path}")

print("Procesando imágenes en la carpeta 'train'...")
for bird_folder in os.listdir(selected_directory):
    bird_dir = os.path.join(selected_directory, bird_folder)
    if os.path.isdir(bird_dir):
        process_images_in_folder(bird_dir)

Se han truncado las últimas 5000 líneas del flujo de salida.
Procesando /content/selected_aves/gbwwre1/XC154154_segment_14.png
Procesando /content/selected_aves/gbwwre1/XC130121_segment_4.png
Procesando /content/selected_aves/gbwwre1/XC127271_segment_2.png
Procesando /content/selected_aves/gbwwre1/XC335419_segment_40.png
Procesando /content/selected_aves/gbwwre1/XC74764_segment_9.png
Procesando /content/selected_aves/gbwwre1/XC124466_segment_5.png
Procesando /content/selected_aves/gbwwre1/XC558292_segment_4.png
Procesando /content/selected_aves/gbwwre1/XC529372_segment_15.png
Procesando /content/selected_aves/gbwwre1/XC532962_segment_18.png
Procesando /content/selected_aves/gbwwre1/XC403205_segment_5.png
Procesando /content/selected_aves/gbwwre1/XC517989_segment_4.png
Procesando /content/selected_aves/gbwwre1/XC472237_segment_3.png
Procesando /content/selected_aves/gbwwre1/XC241529_segment_10.png
Procesando /content/selected_aves/gbwwre1/XC517990_segment_44.png
Procesando /content/sele

In [ ]:
image_paths = []
labels = []
label_map = {}

for bird_folder in os.listdir(selected_directory):
    bird_dir = os.path.join(selected_directory, bird_folder)
    if os.path.isdir(bird_dir):
        # Crear un label map
        if bird_folder not in label_map:
            label_map[bird_folder] = len(label_map)
        label = label_map[bird_folder]
        for file in os.listdir(bird_dir):
            if file.endswith('.png'):
                img_path = os.path.join(bird_dir, file)
                image_paths.append(img_path)
                labels.append(label)

In [ ]:
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
train_paths, val_paths, train_labels, val_labels = train_test_split(train_paths, train_labels, test_size=0.25, random_state=42)

class DataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size=32, dim=(224, 224), n_channels=3, n_classes=10, shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        image_paths_temp = [self.image_paths[k] for k in indexes]
        labels_temp = [self.labels[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(image_paths_temp, labels_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths_temp, labels_temp):
        'Generates data containing batch_size samples'
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, (img_path, label) in enumerate(zip(image_paths_temp, labels_temp)):
            img = Image.open(img_path)
            img = img.resize(self.dim)
            img_array = np.array(img)
            if img_array.shape == (224, 224, 4):  # En caso de tener canal alfa
                img_array = img_array[..., :3]  # Remover el canal alfa
            X[i,] = img_array / 255.0  # Normalizar

            y[i] = label

        return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
from PIL import Image
import numpy as np
import os

# Ruta a la imagen que deseas revisar
img_path = '/content/selected_aves/comrav/XC110257_segment_1.png'  # Reemplaza 'ruta/a/la/imagen.png' con la ruta de una imagen específica

# Verificar si la imagen existe
if os.path.exists(img_path):
    # Abrir la imagen
    img = Image.open(img_path)

    # Convertir la imagen a un array numpy
    img_array = np.array(img)

    # Imprimir detalles de la imagen y su array numpy
    print(f"Formato: {img.format}")
    print(f"Tamaño: {img.size}")
    print(f"Modo: {img.mode}")
    print(f"Valores mínimos del array: {img_array.min()}")
    print(f"Valores máximos del array: {img_array.max()}")
    print(f"Tipo de datos del array: {img_array.dtype}")
else:
    print(f"La imagen en {img_path} no existe.")

Formato: PNG
Tamaño: (224, 224)
Modo: RGB
Valores mínimos del array: 0
Valores máximos del array: 255
Tipo de datos del array: uint8


In [ ]:
# Parámetros
batch_size = 32
epochs = 50
n_classes = len(label_map)

# Crear instancias del generador de datos
train_generator = DataGenerator(train_paths, train_labels, batch_size=batch_size, n_classes=n_classes)
val_generator = DataGenerator(val_paths, val_labels, batch_size=batch_size, n_classes=n_classes)
test_generator = DataGenerator(test_paths, test_labels, batch_size=batch_size, n_classes=n_classes, shuffle=False)

In [ ]:
# Crear la arquitectura de ResNet50 sin pesos preentrenados
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(n_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(train_generator, validation_data=val_generator, epochs=epochs)

Epoch 1/50
853/853 [==============================] - 380s 397ms/step - loss: 0.8684 - accuracy: 0.7214 - val_loss: 2.1178 - val_accuracy: 0.3020
Epoch 2/50
853/853 [==============================] - 345s 405ms/step - loss: 0.4644 - accuracy: 0.8516 - val_loss: 0.7522 - val_accuracy: 0.7717
Epoch 3/50
853/853 [==============================] - 341s 399ms/step - loss: 0.3455 - accuracy: 0.8869 - val_loss: 0.5974 - val_accuracy: 0.8175
Epoch 4/50
439/853 [==============>...............] - ETA: 2:26 - loss: 0.2511 - accuracy: 0.9179

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')